# Discrete Response Models
### Vijay Ivaturi, Chris Rackauckas
### July 19th, 2019
# Introduction

In this tutorial we will go over the simulation of discrete responses. Many
pharmacometrics scenarios have observables, such as pain scores or counts,
which necessarily have to be discrete. Handling this discreteness can be
paramount to getting an appropriate data fit and to properly understand the
variation.

Luckily, in Pumas, discrete outputs are handled no differently from the rest
of the Pumas toolchain. In Pumas, to have a discrete distribution as output,
simply have that your `derived` or `observed` variables come from a discrete
distribution like a `Poisson` process.

## Binary Response Example

First, let's take a look at a binary response. A binary response is a model
which gives an output of 0 or 1 with a probability `p`. In Pumas, this is
represented by a `Bernoulli(p)` distribution.

To get started, first let's load `Pumas` and read in some example data:

In [ ]:
using Pumas, StatsFuns
data = read_pumas(example_nmtran_data("pain_remed"),
    cvs = [:arm, :dose, :conc, :painord,:remed],event_data=false)

Next let's implement a model with a binary response. Here, we do not have an
`@dynamics` porition. Pumas will automatically handle this. In our `derived`,
we define a `logistic` from [StatsFuns](https://github.com/JuliaStats/StatsFuns.jl#basic-functions)

In [ ]:
import StatsFuns.logistic
binary_model = @model begin
    @param begin
        intercept ∈ RealDomain(init=0.001)
        tvslope ∈ RealDomain(init=0.0001)
        Ω ∈ VectorDomain(1)
    end

    @random begin
        η ~ MvNormal(Ω)
    end

    @covariates arm dose

    @pre begin
        rx = dose > 0 ? 1 : 0
        slope = tvslope*rx
        logit = intercept + slope + η[1]
    end

    @derived begin
        pain = logistic(logit)
        dv ~ Bernoulli(logistic(logit))
    end
end

Note that we could have alternatively defined our `@pre` like:

In [ ]:
@pre begin
    logit = intercept + tvslope*(dose > 0 ? 1 : 0) + η[1]
    logit = intercept + tvslope*Int(dose > 0) + η[1]
end

more directly instead of using `logistic`.

Now let's fit our model to the data:

In [ ]:
param = (
    intercept = 0.001,
    tvslope = 0.0001,
    Ω = [1.0]
    )
res = fit(binary_model,data,param,Pumas.LaplaceI())

and simulate some outputs:

In [ ]:
sim = simobs(binary_model,data,res.param)
simdf = DataFrame(sim, include_events=false)
first(simdf,6) # Print only the first 6 rows

Note that now our simulation output for `dv` is true/false values pulled with
probability given by `logit` dependent on the individual's random effects.

## Poisson Response Example

Next let's use a `Poisson` counting process in our model. Here we generate a
population where everyone is receiving the same doses as a covariate.

In [ ]:
pop = Population(map(i -> Subject(id=i,cvs=(dose = i.*10,),time=[0.0]),1:10))

Now we define our model without dynamics, and directly use the dose information
to predict the count for some observable `dv`:

In [ ]:
poisson_model = @model begin
  @param begin
    tvbase ∈ RealDomain(init=3.0, lower=0.1)
    d50 ∈ RealDomain(init=50, lower=0.1)
    Ω  ∈ PSDDomain(fill(0.1, 1, 1))
  end

  @random begin
    η ~ MvNormal(Ω)
  end

  @pre begin
    baseline = tvbase*exp(η[1])
  end

  @covariates dose

  @derived begin
    dv ~ @. Poisson(baseline*(1-dose/(dose + d50)))
  end
end

and simulate runs from the model:

In [ ]:
sim = simobs(poisson_model,pop)
simdf = DataFrame(sim, include_events=false)

Here `dv` is an integer output probabilistically dependent on the dose.

In [ ]:
using PumasTutorials
PumasTutorials.tutorial_footer(WEAVE_ARGS[:folder],WEAVE_ARGS[:file])